In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime
import altair as alt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
type(datetime.datetime)

In [ ]:
c_df=pd.read_csv('/kaggle/input/covid19-in-india/covid_19_india.csv')
c_df.Date=pd.to_datetime(c_df.Date)
c_df.shape

In [ ]:
c_df.info()

In [ ]:
# aggreagate smaller states
n=15
topStates=c_df[['State/UnionTerritory','Confirmed']].groupby(['State/UnionTerritory']).max().sort_values('Confirmed')[-n:].index.values
c_df.loc[~c_df['State/UnionTerritory'].isin(topStates),['State/UnionTerritory']]='Other'
for f in ['ConfirmedIndianNational','ConfirmedForeignNational']:
    c_df[f]=c_df[f].str.replace("-","0").astype(int)

In [ ]:
c_df=c_df.groupby(['Date','State/UnionTerritory']).sum().reset_index().set_index('Sno')


In [ ]:
c_df.columns

In [ ]:
#trend since beginning
total=alt.Chart(c_df[['Date',]+cols_].groupby('Date').agg('max').reset_index()
                  ).mark_line().transform_fold( cols_
).transform_calculate(
    active='datum.Confirmed - datum.Cured'
).encode(
    alt.Y('active:Q'),
#     alt.Color('key:N'),
    alt.X('Date')
)
alt.Chart(c_df[['Date','State/UnionTerritory']+cols_].groupby(['Date','State/UnionTerritory']).agg('max').reset_index()
                  ).mark_line(
).transform_fold( 
    cols_#, groupby=['State/UnionTerritory']
).transform_calculate(
    active='datum.Confirmed - datum.Cured'
).encode(
    alt.Y('active:Q'),
    alt.Color('State/UnionTerritory:N'),
    alt.X('Date:T'),
    tooltip=['active:Q','Date:T','State/UnionTerritory:N']
).interactive()

In [ ]:
since = datetime.datetime.now() - pd.to_timedelta("30 day")

cols_=['Cured','Deaths','Confirmed']
df_=c_df[c_df.Date >= since ][['Date','State/UnionTerritory']+cols_]#.set_index(['State/UnionTerritory','Date'])
baseData=df_[['Date','State/UnionTerritory']].groupby('State/UnionTerritory').agg('min')
baseData=baseData.merge(df_[['Date','State/UnionTerritory']+cols_]
               ,on=['Date','State/UnionTerritory'])
baseData=baseData.set_index('State/UnionTerritory')
df_=df_.join(baseData[cols_],
         how='left',
         on=['State/UnionTerritory'],
         rsuffix="_base")

for f in cols_:
     df_[f+'_g']=df_[f]/df_[f+"_base"]*100-100


In [ ]:
ST_sel=alt.selection_multi(name='ST',fields=['State/UnionTerritory'])
# SL_sel = alt.selection_multi(name="SL",fields=['Service_Line'])
ch1=alt.Chart(df_[['Date','State/UnionTerritory']+cols_].groupby('State/UnionTerritory').agg('max').reset_index()
                  ).mark_bar().transform_fold( cols_
).encode(
    alt.X('value:Q'),
    alt.Color('key:N'),
    alt.Y('State/UnionTerritory')
).add_selection(
ST_sel
)

growth=alt.Chart(df_).mark_line().encode(
    alt.X('Date:T'),
    alt.Y('Confirmed_g:Q'),
    color='State/UnionTerritory:N',
    size=alt.value(5),
    opacity=alt.condition(ST_sel, alt.value(1), alt.value(.3)),
    tooltip=list(df_.columns)
).add_selection(
ST_sel
)

alt.hconcat(ch1 , growth)

In [ ]:
alt.data_transformers.enable(max_rows=None)


alt.Chart(df_).mark_line().encode(
    alt.X('Date:T'),
    alt.Y('Confirmed_g:Q'),
    color='State/UnionTerritory:N',
    size=alt.value(5),
    opacity=alt.value(.5),
    tooltip=list(df_.columns)
).interactive()